# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [ ]:
import numpy as np
import pandas as pd
import torch
from pinn import PINN
import utils

In [ ]:
in_dir = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"
norms_filename = "front_wing_norms_final.csv"

out_dir = "/Users/ggito/repos/pinns/data/"
models_out_dir = "/Users/ggito/repos/pinns/data/models/"
log_out_dir = "/Users/ggito/repos/pinns/data/log/"

In [ ]:
if torch.backends.mps.is_available():
  device = torch.device("mps")
elif torch.cuda.is_available():
  device = torch.device("cuda")
else:
  print("GPU device not found.")

print(device)

In [ ]:
wing_df = pd.read_csv(in_dir + points_filename)
# norm_df = pd.read_csv(in_dir + norms_filename)

In [ ]:
# Density (rho): 1.2041kg/m^3
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1
mu = 1

# m/s
in_velocity = 1

# Domain limits
x_max = 1
y_max = 1
z_max = 1
t_max = 2

In [ ]:
input_dim = 4
output_dim = 4
# hidden_units = [2058, 2058, 2058]
hidden_units = [1024, 1024, 1024]

pinn = PINN(input_dim, output_dim, hidden_units).to(device)

In [ ]:
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=1)

epochs = 300

Nf = 20000   # num of collocation points -> pde evaluation
N0 = 20000   # num of points to evaluate initial conditons
Nb = 20000   # num of points to evaluate boundary conditions
Nw = 20000   # num of points of the surface of the front wing to evaluate boundary conditions

c1 = 1
c2 = 1
c3 = 1
c4 = 1

In [ ]:
epoch_loss = []
checkpoint_epochs = 2
filename = "v3"

In [ ]:
try: 
  pinn.train_pinn(epochs=epochs, 
                optimizer=optimizer, 
                wing_df=wing_df,
                Nf=Nf, 
                N0=N0, 
                Nb=Nb, 
                Nw=Nw, 
                x_max=x_max, 
                y_max=y_max, 
                z_max=z_max, 
                t_max=t_max, 
                c1=c1, 
                c2=c2, 
                c3=c3, 
                c4=c4, 
                in_velocity=in_velocity,
                mu=mu, 
                rho=rho, 
                device=device,
                checkpoint_epochs=checkpoint_epochs, 
                models_out_dir=models_out_dir, 
                filename=filename, 
                log_out_dir=log_out_dir)

except KeyboardInterrupt:
    print("Training stopped by user.")

In [ ]:
pinn, optimizer = utils.load_checkpoint(pinn, optimizer, models_out_dir + filename + "_" + str(2) + ".pt")
# for param_group in optimizer.param_groups:
#     param_group['lr'] = 0.01

# for param_group in optimizer.param_groups:
#   print(param_group['lr'])  # Should print the new learning rate

In [ ]:
pinn.print_all_metrics()